In [1]:
import yfinance as yf
from datetime import datetime, timedelta
from prettytable import PrettyTable
import os
from pandas_market_calendars import get_calendar
import pandas as pd
import re
import time


# Define the path to the CSV file containing 'Client Name'
csv_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\FII Data\FII_Reconciliation\FII_LIST_240128122722.csv'

# Read 'Client Name' from the CSV file
client_name_df = pd.read_csv(csv_path)
client_names = client_name_df['Client Name'].tolist()

# Define the path to the folder containing Excel files
excel_folder_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\OutboundSuperstars'

# Define the start and end dates for the file date range
start_date = datetime.strptime('20230601', "%Y%m%d")
end_date = datetime.strptime('20240208', "%Y%m%d")

# Create a PrettyTable to store the results
results_table = PrettyTable()
results_table.field_names = ['Code', 'Client Name', 'Holding', 'Percent Holding', 'Investor Type', 'File Date']

# Create a list to store matching rows
matching_rows = []

# Iterate through each Excel file
for excel_file in os.listdir(excel_folder_path):
    excel_file_path = os.path.join(excel_folder_path, excel_file)

    # Get the list of sheet names in the Excel file
    sheet_names = pd.ExcelFile(excel_file_path).sheet_names

    # Check if 'Shareholding' sheet exists
    if 'Shareholding' in sheet_names:
        # Read the 'Shareholding' sheet
        df = pd.read_excel(excel_file_path, sheet_name='Shareholding', skiprows=6)

        # Extract date from the file name using regular expression
        match = re.search(r'\d{8}', excel_file)  # Assuming the date in the file name is in YYYYMMDD format
        if match:
            file_date_str = match.group(0)
            file_date = datetime.strptime(file_date_str, "%Y%m%d")

            # Check if the file date is within the specified range
            if start_date <= file_date <= end_date:
                # Check if 'Client Name' is present in the DataFrame
                if 'Client Name' in df.columns:
                    # Check if each 'Client Name' from the CSV is present in the Excel file
                    present_in_excel = [client_name in df['Client Name'].values for client_name in client_names]

                    # Display only the specified columns for matched records
                    for i, present in enumerate(present_in_excel):
                        if present:
                            matching_row = df[df['Client Name'] == client_names[i]]
                            matching_row = matching_row[['Code', 'Client Name', 'Holding', 'Percent Holding', 'Investor Type']]
                            matching_row['File Date'] = file_date_str
                            matching_rows.append(matching_row)

# Concatenate all matching rows into a DataFrame
results_df = pd.concat(matching_rows)

# Sort the DataFrame by 'File Date' in descending order
results_df = results_df.sort_values(by='File Date', ascending=False)

# Print the results using PrettyTable
for _, row in results_df.iterrows():
    results_table.add_row(row.tolist())



from datetime import datetime, timedelta

def is_working_day(date_str):
    # Parse the date string to datetime format
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Check if the date falls on a weekend (Saturday or Sunday)
    if date_obj.weekday() >= 5:
        return False
    
    # Check if the date falls on a special working day
    special_working_days = {"2024-01-27", "2024-03-02"}  # Example: Christmas Eve
    if date_str in special_working_days:
        return True
    
    # Check if the date falls on a holiday
    # Add your holiday logic here
    holidays = {"2023-01-26", "2023-03-07", "2023-03-30", "2023-04-04", "2023-04-07", 
                "2023-04-14", "2023-05-01", "2023-06-29", "2023-07-29", "2023-08-15", 
                "2023-09-19", "2023-10-02", "2023-10-24", "2023-11-14", "2023-11-27", 
                "2023-12-25","2024-01-22",
                "2024-01-26", "2024-03-08", "2024-03-25", "2024-03-29", "2024-04-11", 
                "2024-04-17", "2024-05-01", "2024-06-17", "2024-07-17", "2024-08-15", 
                "2024-10-02", "2024-11-01", "2024-11-15", "2024-12-25"}
    if date_str in holidays:
        return False
    
    # If the date is not a weekend, holiday, or special working day, it's a regular working day
    return True

def is_working_day_if_not_next_working_date(date):
    # Check if the input is already a string
    if isinstance(date, str):
        # Parse the date string to datetime format
        date_obj = datetime.strptime(date, '%Y-%m-%d')
    else:
        # Use the provided Timestamp object
        date_obj = date
    
    # Check if the date is a working day
    if is_working_day(date_obj.strftime('%Y-%m-%d')):
        return date_obj
    
    # If the date is not a working day, find the next working date
    while True:
        date_obj += timedelta(days=1)
        if is_working_day(date_obj.strftime('%Y-%m-%d')):
            return date_obj




def download_stock_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

def calculate_percentage_changes(data, stock_code, start_date,client_name,holding,percent_holding,investor_type,file_date ):
    result_dict = {
        "Stock Code": stock_code,
        "Client Name": client_name,  # Corrected variable name
        "Holding": holding,  # Corrected variable name
        "Percent Holding": percent_holding,  # Corrected variable name
        "Investor Type": investor_type,  # Corrected variable name
        "File Date": file_date,  # Corrected variable name
    }


    periods = [1, 20, 40, 60]

    for period in periods:
         if len(data) >= period + 1:

            close_change_1d = volume_change = None
    
            # Check if the start date is in the index
            if start_date in data.index:
                close_change_1d = ((data['Close'].iloc[period] - data['Close'].loc[start_date]) / data['Close'].loc[start_date]) * 100
                volume_change = ((data['Volume'].iloc[period] - data['Volume'].loc[start_date]) / data['Volume'].loc[start_date]) * 100
    
            result_dict[f"{period}D P (%)"] = f"{close_change_1d:.2f}" if close_change_1d is not None else "nan"
            result_dict[f"{period}D V (%)"] = f"{volume_change:.2f}" if volume_change is not None else "nan"

    return result_dict



# Assuming results_df has a 'File Date' column in '%Y%m%d' format
results_df['File Date'] = pd.to_datetime(results_df['File Date'], format='%Y%m%d')



# Create a PrettyTable for displaying the results
table = PrettyTable()
header = ["Stock Code", "Client Name","Holding","Percent Holding","Investor Type","File Date"]

for period in [1, 20, 40, 60]:
    header.append(f"{period}D P (%)")
    header.append(f"{period}D V (%)")
table.field_names = header

# Create a raw table to display original data
raw_table = PrettyTable()
raw_table.field_names = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Stock Code"]



for _, row in results_df.iterrows():
    
    # Assuming results_df has a 'File Date' column in '%Y%m%d' format
    file_date = pd.to_datetime(row['File Date'], format='%Y%m%d')
    
    # Define start date
    start_date = file_date  # Select the file date as the start date

    # while True:
    #     # Find the next day
    start_date_pass = is_working_day_if_not_next_working_date(start_date)
    print(start_date_pass)
    # Calculate end date for historical data (two months later)
    end_date = (start_date_pass + timedelta(days=120)).strftime("%Y-%m-%d")

    # Download stock data for the next day
    if isinstance(row['Code'], str):
        stock_data = download_stock_data(row['Code'] + ".NS", start_date_pass, end_date)
  
        percentage_changes = calculate_percentage_changes(stock_data, row['Code'], start_date_pass, row['Client Name'], row['Holding'], row['Percent Holding'], row['Investor Type'], file_date)
    
        # Add the result to the PrettyTable
        table.add_row([percentage_changes.get(field, "nan") for field in table.field_names])


print("\nPercentage Changes:")
print(table)



2024-02-08 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-08 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-07 00:00:00
[*********************100%%**********************]  1 of 1 completed

C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



2024-02-07 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-07 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-07 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-06 00:00:00
[*********************100%%**********************]  1 of 1 completed

C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex 


2024-02-06 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-06 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-02-06 00:00:00
[*********************100%%**********************]  1 of 1 completed

...

[*********************100%%**********************]  1 of 1 completed
2023-06-02 00:00:00
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['526025.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')



2023-06-02 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-06-01 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-06-01 00:00:00
[*********************100%%**********************]  1 of 1 completed

Percentage Changes:


C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Stock Code |                         Client Name                          |    Holding     | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) |
+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|  YATHARTH  |                 Hsbc Global Investment Funds                 |  Above 1% now  |       1.06      |      FII      | 2024-02-08 00:00:00 |   3.61   |  -23.05  |    nan    |    nan    |    nan    |    nan    |    nan    |    nan    |
|   BAJEL    |      

In [ ]:
Filter the percentage of increase

In [6]:
# Filter rows where '10D V (%)' is greater than 5
filtered_results = [row for row in table._rows if float(row[8]) > 10]

# Create a new PrettyTable for displaying the filtered results
filtered_table = PrettyTable()
filtered_table.field_names = table.field_names

# Add the filtered rows to the new table
for row in filtered_results:
    filtered_table.add_row(row)

# Display the filtered PrettyTable
print("\nFiltered Results (60D P (%) > 50):")
print(filtered_table)



Filtered Results (60D P (%) > 50):
+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Stock Code |                         Client Name                          |    Holding     | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) |
+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|  RELINFRA  |                        Vanguard Fund                         |  below 1% now  |       nan       |      FII      | 2024-01-20 00:00:00 |   2.42   |  -26.88  |   11.83   |   40.43   |    nan    |    nan    |    nan  

In [7]:
from prettytable import PrettyTable
import math  # Import math module to check for NaN

# Assuming 'table' is the original PrettyTable
# Assuming the columns in the table are indexed as follows: 
# 0: Stock Code, 1: Client Name, 2: Holding, 3: Percent Holding, 
# 4: Investor Type, 5: File Date, 6: 1D P (%), 7: 1D V (%), 
# 8: 20D P (%), 9: 20D V (%), 10: 40D P (%), 11: 40D V (%), 
# 12: 60D P (%), 13: 60D V (%), 14: 120D P (%), 15: 120D V (%)

# Copy the table to a new table
new_table = PrettyTable()
new_table.field_names = table.field_names
for row in table._rows:
    new_table.add_row(row)

# Filter rows where '10D V (%)' is greater than 50 and apply the rating logic
filtered_results = []
print("Length of table:", len(new_table._rows))

for row in new_table._rows:
    # Convert row[10] and row[12] to float if they are not NaN
    p_40d = float(row[10])
    p_60d = float(row[12])
    
    # Check if p_40d and p_60d are not NaN
    if not math.isnan(p_40d) and not math.isnan(p_60d):
        if 'Above' in row[2] and p_60d > 0:
            row.append(1)
        elif 'Above' in row[2] and p_60d < 0:
            row.append(2)
        elif 'Bought' in row[2] and p_60d > 0:
            row.append(1)
        elif 'Bought' in row[2] and p_60d < 0:
            row.append(2)
        elif 'Sold' in row[2] and p_60d > 0:
            row.append(2)
        elif 'Sold' in row[2] and p_60d < 0:
            row.append(1)
        elif 'below' in row[2] and p_60d > 0:
            row.append(2)
        elif 'below' in row[2] and p_60d < 0:
            row.append(1)
        else:
            row.append(None)
        filtered_results.append(row)

# Create a new PrettyTable for displaying the filtered results
filtered_table = PrettyTable()
filtered_table.field_names = new_table.field_names + ['Rating']  # Append 'Rating' column

# Add the filtered rows to the new table
for row in filtered_results:
    # Ensure the row has the same number of values as the original table
    if len(row) == len(new_table.field_names) + 1:
        filtered_table.add_row(row)

# Display the filtered PrettyTable
print("\nFiltered Results (60D P (%) > 50):")
print(filtered_table)


Length of table: 1439

Filtered Results (60D P (%) > 50):
+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+
| Stock Code |                         Client Name                          |    Holding     | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) | Rating |
+------------+--------------------------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+
|   360ONE   |                        Vanguard Fund                         |  Above 1% now  |       1.02      |      FII      | 2023-11-29 00:00:00 |   2.40   |  141.26  |   26.83

In [8]:
from prettytable import PrettyTable
from collections import defaultdict

# Initialize a nested defaultdict to store the counts for each client name and rating
client_rating_counts = defaultdict(lambda: defaultdict(int))

# Group the filtered results by "Client Name" and "Rating" and count the occurrences
for row in filtered_results:
    client_name = row[1]  # Assuming "Client Name" is at index 1
    rating = row[-1]  # Assuming "Rating" is the last element in the row
    client_rating_counts[client_name][rating] += 1

# Create a PrettyTable for displaying the counts
count_table = PrettyTable()
count_table.field_names = ["Client Name", "Rating", "Times"]

# Add the counts to the table
for client_name, ratings in client_rating_counts.items():
    for rating, count in ratings.items():
        count_table.add_row([client_name, rating, count])

# Display the PrettyTable
print("Consolidated Counts:")
print(count_table)


Consolidated Counts:
+--------------------------------------------------------------+--------+-------+
|                         Client Name                          | Rating | Times |
+--------------------------------------------------------------+--------+-------+
|                        Vanguard Fund                         |   1    |   26  |
|                        Vanguard Fund                         |   2    |   18  |
|                Government Pension Fund Global                |   1    |   92  |
|                Government Pension Fund Global                |   2    |   62  |
|            Massachusetts Institute Of Technology             |   2    |   29  |
|            Massachusetts Institute Of Technology             |   1    |   9   |
|            Elara India Opportunities Fund Limited            |   2    |   9   |
|            Elara India Opportunities Fund Limited            |   1    |   4   |
|                Abu Dhabi Investment Authority                |   2    |   3

In [ ]:
!Important Filter specific stock

In [27]:
from prettytable import PrettyTable

# Assuming table contains the data with the structure provided

# # Check if the data is loaded correctly
# print("Data loaded into table:")
# for row in table._rows:
#     print(row)

# Filter rows where stock code is 'Cupid' or 'Suzlon'
filtered_results = [row for row in table._rows if row[0] in ['STAR']]
# filtered_results = [row for row in table._rows if row[1] in ['Smallcap World Fund Inc']]


# Create a new PrettyTable for displaying the filtered results
filtered_table = PrettyTable()
filtered_table.field_names = table.field_names

# Add the filtered rows to the new table
for row in filtered_results:
    filtered_table.add_row(row)

# Display the filtered PrettyTable
print("\nFiltered Results for Stock Codes 'Cupid' and 'Suzlon':")
print(filtered_table)



Filtered Results for Stock Codes 'Cupid' and 'Suzlon':
+------------+--------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Stock Code |             Client Name              |    Holding     | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) |
+------------+--------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|    STAR    | Morgan Stanley Asia (Singapore) Pte. | Bought 0.02000 |       1.08      |      FII      | 2024-01-19 00:00:00 |   0.00   | -100.00  |    nan    |    nan    |    nan    |    nan    |    nan    |    nan    |
|    STAR    | Morgan Stanley Asia (Singapore) Pte. |  Above

In [ ]:
!Important Filter specific stock

In [24]:
from prettytable import PrettyTable

# Assuming you have defined 'table' containing the original data

# Filter rows where '10D V (%)' is greater than 40 and 'Holding' column contains 'Above' or 'Bought'
filtered_results = []
for row in table._rows:
    # if float(row[12]) > 10 and ('Above' in row[2] or 'Bought' in row[2] ) and ('Smallcap World Fund Inc' in row[1]):
    # if float(row[12]) > 10 and ('Vanguard Fund' in row[1]):
    # if ('Vanguard Fund' in row[1]):
    # if float(row[12]) > 10 and ('Above 1% now' in row[2] and float(row[3]) > 2 ) and ('Smallcap World Fund Inc' in row[1]):
    # if float(row[12]) > 10 and ('Above 1% now' in row[2] and float(row[3]) > 2 and float(row[10]) > 30):
    if float(row[8]) > 10 and ('Above 1% now' in row[2] and float(row[3]) > 2 ):

        filtered_results.append(row)

# Create a new PrettyTable for displaying the filtered results
filtered_table = PrettyTable()
filtered_table.field_names = table.field_names

# Add the filtered rows to the new table
for row in filtered_results:
    filtered_table.add_row(row)

# Display the filtered PrettyTable
print("\nFiltered Results (60D P (%) > 40 and Holding contains 'Above' or 'Bought'):")
print(filtered_table)



Filtered Results (60D P (%) > 40 and Holding contains 'Above' or 'Bought'):
+------------+------------------------------------------+--------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Stock Code |               Client Name                |   Holding    | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) |
+------------+------------------------------------------+--------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   CREST    |          Elm Park Fund Limited           | Above 1% now |       4.52      |      FII      | 2024-01-04 00:00:00 |   2.21   |  64.21   |   62.89   |   201.67  |    nan    |    nan    |    nan    |    nan    |
|  HINDALCO  |         Governme

In [3]:
import numpy as np
import pandas as pd
import textwrap
from pymongo import MongoClient
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.sparse import hstack
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(filename='news_analysis.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip01022024.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        collection = client[database_name][collection_name]

        # Query all records in the collection
        start_date = datetime(2024, 2, 2, 0, 0, 0)
        end_date = datetime(2024, 2, 3, 0, 0, 0)
        find_query = {
            "created_at": {
                "$gte": start_date,
                "$lt": end_date
            }
        }

        all_records = collection.find(find_query)
        mongo_documents = list(all_records)

        if not mongo_documents:
            logging.warning("No records found from DB")
            print("No records found from DB")
        else:
            # Read the "Portfolio" sheet from the Excel file
            portfolio_df = pd.read_excel(excel_file_path, sheet_name='Top1000NSE', header=0)

            # Assuming the column containing stock names is named "Top1000NSE"
            stock_names_in_portfolio = portfolio_df['Top1000NSE'].tolist()

            # Create a list to store tuples of (Scrip, paragraph_content, label, confidence, date, strength)
            result_list = []

            # Check if each stock name in the portfolio is present in MongoDB documents
            for document in mongo_documents:
                for stock_name in stock_names_in_portfolio:
                    if stock_name.lower() in document['paragraph_content'].lower():
                        # Wrap paragraph content to a maximum of five words per line
                        wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=110))
                        # Extract the numeric part from the 'Score' column
                        numeric_score = int(''.join(filter(str.isdigit, document['sentiment']['label'])))

                        result_list.append((
                            stock_name,
                            wrapped_content,
                            numeric_score,  # Use the extracted numeric score
                            document['sentiment']['confidence'],
                            document['created_at'].strftime('%Y-%m-%d'),  # Extract date from created_at
                            document['FinBertScore'],
                            None  # Placeholder for Strength column
                        ))
                        break  # Break if at least one match is found for the current document

            # Create a DataFrame from the result list
            result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence', 'Date', 'FinBertScore', 'Strength'])

            # Check if Positive/Negative sheet is present in the Excel file
            xls = pd.ExcelFile(excel_file_path, engine='openpyxl')  # Use openpyxl engine
            if 'PositiveNegative' in xls.sheet_names:
                # Read the Positive/Negative sheet from the Excel file
                positive_negative_df = pd.read_excel(xls, sheet_name='PositiveNegative', header=0)

                # Assuming the column containing positive/negative words is named "PositiveNegative"
                positive_negative_words = positive_negative_df['PositiveNegative'].tolist()

                # Update the Strength column based on positive/negative words
                result_df['Strength'] = result_df.apply(
                    lambda row: ', '.join(
                        positive_negative_word
                        for positive_negative_word in positive_negative_words
                        if positive_negative_word.lower() in row['Paragraph_content'].lower()
                    ),
                    axis=1
                )

                # Wrap the entire 'Strength' column to a maximum width of 140 characters
                result_df['Strength'] = result_df['Strength'].apply(lambda x: '\n'.join(textwrap.wrap(str(x), width=17)))

            # Sort the DataFrame based on multiple columns in ascending order
            result_df['Confidence'] = result_df['Confidence'].apply(lambda x: f"{x:.5f}")
            result_df = result_df.sort_values(by=['Date', 'Scrip', 'Score', 'Confidence'], ascending=[False, True, False, False])

            # Display the DataFrame
            print(tabulate(result_df, headers='keys', tablefmt='pretty'))

        
except Exception as e:
    logging.error(f"Error: {e}")
    print(e)


+----+--------------------------+----------------------------------------------------------------------------------------------------------------+-------+------------+------------+--------------+------------------+
|    |          Scrip           |                                               Paragraph_content                                                | Score | Confidence |    Date    | FinBertScore |     Strength     |
+----+--------------------------+----------------------------------------------------------------------------------------------------------------+-------+------------+------------+--------------+------------------+
| 42 |       Adani Ports        | Shares of Adani Ports and SEZ Ltdjumped over 4.6% in early trade to ₹1,275. The company reported a 65 per cent |   1   |  0.51959   | 2024-02-02 |      1       |                  |
|    |                          | rise in consolidated net profit in the December quarter. The company’s revenue increased 45 per cent year-

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import re
from prettytable import PrettyTable











# Assuming you have a DataFrame with features and labels for the first dataset
data1 = {
    'Date': ['2021-01-01', '2021-01-02', '2021-01-03'],
    'Stock': ['ABC', 'XYZ', 'ABC'],
    'FinBertScore': [1, 2, 0],
    'PriceMovement': [1, 0, 1]  # 1 for increase, 0 for decrease
}

# Assuming you have a DataFrame for the second dataset
data2 = {
    'Date': ['2021-01-01', '2021-01-02'],
    'Stock': ['ABC', 'XYZ'],
    'FII Name': ['XYZ FII', 'ABC FII'],
    'FII Action': ['Buy 3%', 'Sell 2%'],
    'Future Price Movement Weekly': ['+1%', '-0.5%'],
    'Future Price Movement Monthly': ['+4%', '-2%']
}

# Create DataFrames from the provided data
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Feature selection for the first dataset
features1 = ['FinBertScore']
target1 = 'PriceMovement'

# Feature selection for the second dataset
features2 = ['FII Action']
target2 = 'Future Price Movement Weekly'

# Merge the datasets on 'Date' and 'Stock'
df_combined = pd.merge(df1, df2, on=['Date', 'Stock'], how='inner')

# Check if the merged DataFrame is not empty
if df_combined.empty:
    print("Merged DataFrame is empty. Check merging columns and common values.")
else:
    # Extract numerical information from 'FII Action' column
    df_combined['FII Action'] = df_combined['FII Action'].apply(lambda x: re.search(r'\d+', x).group()).astype(float)

    # Create features and labels for the combined dataset
    X_combined = df_combined[features1 + features2]
    y_combined = df_combined[target1]

    # Split the data into training and testing sets
    X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
        X_combined, y_combined, test_size=0.2, random_state=42, shuffle=True
    )
    # Convert NumPy arrays back to DataFrames
    X_train_combined = pd.DataFrame(X_train_combined, columns=X_combined.columns, index=y_train_combined.index)
    X_test_combined = pd.DataFrame(X_test_combined, columns=X_combined.columns, index=y_test_combined.index)
    
    # Choose a classification algorithm (Random Forest in this example)
    model_combined = RandomForestClassifier(random_state=42)
    
    # Train the model on the combined dataset
    model_combined.fit(X_train_combined, y_train_combined)
    
    # Make predictions on the combined test set
    y_pred_combined = model_combined.predict(X_test_combined)
    
    # Map predicted values to Buy (1) or Sell (0)
    buy_sell_predictions_combined = ['Buy' if pred == 1 else 'Sell' for pred in y_pred_combined]
    
    # Create a new column 'Buy/Sell' in the combined test set
    df_test_combined = df_combined.loc[X_test_combined.index].copy()
    df_test_combined['Buy/Sell'] = buy_sell_predictions_combined
    

    # Display the DataFrame with Buy/Sell column for the combined dataset in a pretty table
    table = PrettyTable()
    table.field_names = df_test_combined.columns
    
    for _, row in df_test_combined.iterrows():
        table.add_row(row)
    
    print("Pretty Table with Buy/Sell for the combined dataset:")
    print(table)

Pretty Table with Buy/Sell for the combined dataset:
+------------+-------+--------------+---------------+----------+------------+------------------------------+-------------------------------+----------+
|    Date    | Stock | FinBertScore | PriceMovement | FII Name | FII Action | Future Price Movement Weekly | Future Price Movement Monthly | Buy/Sell |
+------------+-------+--------------+---------------+----------+------------+------------------------------+-------------------------------+----------+
| 2021-01-02 |  XYZ  |      2       |       0       | ABC FII  |    2.0     |            -0.5%             |              -2%              |   Buy    |
+------------+-------+--------------+---------------+----------+------------+------------------------------+-------------------------------+----------+
